<a href="https://colab.research.google.com/github/stepstardream/graph-neural-network/blob/main/mtad_gat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://s3-us-west-2.amazonaws.com/telemanom/data.zip

--2023-04-09 04:49:14--  https://s3-us-west-2.amazonaws.com/telemanom/data.zip
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.219.56, 52.92.241.136, 52.92.213.136, ...
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.219.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85899803 (82M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  81.92M  35.6MB/s    in 2.3s    

2023-04-09 04:49:17 (35.6 MB/s) - ‘data.zip’ saved [85899803/85899803]



In [2]:
!unzip -qq data.zip

In [3]:
!git clone https://github.com/ML4ITS/mtad-gat-pytorch.git

Cloning into 'mtad-gat-pytorch'...
remote: Enumerating objects: 6170, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 6170 (delta 3), reused 2 (delta 0), pack-reused 6161
Receiving objects: 100% (6170/6170), 920.66 MiB | 34.66 MiB/s, done.
Resolving deltas: 100% (2700/2700), done.
Updating files: 100% (158/158), done.


In [4]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
import torch.nn as nn
from tqdm.notebook import tqdm

In [5]:
smap_channels = pd.read_csv('mtad-gat-pytorch/datasets/data/smap_train_md.csv')
print("Number of Channels : ", len(smap_channels))
smap_channels.head(5)
smap_channels = list(smap_channels['chan_id'].values)
len(smap_channels)

Number of Channels :  53


53

In [6]:
smap_data = []
for smap_channel in smap_channels:
    tmp_data = np.load(os.path.join('data/train/', smap_channel + '.npy'))
    smap_data.extend(tmp_data)
    
smap_data = np.array(smap_data)
print("Shape of SMAP Data : ", smap_data.shape)

Shape of SMAP Data :  (135183, 25)


In [7]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler
def normalize_data(data, scaler=None):
    data = np.asarray(data, dtype=np.float32)
    if np.any(sum(np.isnan(data))):
        data = np.nan_to_num(data)

    if scaler is None:
        scaler = MinMaxScaler()
        scaler.fit(data)
    data = scaler.transform(data)
    print("Data normalized")

    return data, scaler

In [8]:
smap_data_norm, scaler = normalize_data(smap_data)
smap_data_pt = torch.from_numpy(smap_data)
smap_data_pt.size()

Data normalized


torch.Size([135183, 25])

In [9]:
class SlidingWindowDataset(Dataset):
    def __init__(self, data, window, target_dim=None, horizon=1):
        self.data = data
        self.window = window
        self.target_dim = target_dim
        self.horizon = horizon

    def __getitem__(self, index):
        x = self.data[index : index + self.window]
        y = self.data[index + self.window : index + self.window + self.horizon]
        return x, y

    def __len__(self):
        return len(self.data) - self.window

In [10]:
Window = 100
BatchSZ =  256
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [11]:
smap_x_y = SlidingWindowDataset(smap_data_pt, 100)

In [12]:
def create_data_loaders(train_dataset, batch_size, val_split=0.1, shuffle=True, test_dataset=None):
    train_loader, val_loader, test_loader = None, None, None
    if val_split == 0.0:
        print(f"train_size: {len(train_dataset)}")
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

    else:
        dataset_size = len(train_dataset)
        indices = list(range(dataset_size))
        split = int(np.floor(val_split * dataset_size))
        if shuffle:
            np.random.shuffle(indices)
        train_indices, val_indices = indices[split:], indices[:split]

        train_sampler = SubsetRandomSampler(train_indices)
        valid_sampler = SubsetRandomSampler(val_indices)

        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
        val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=valid_sampler)

        print(f"train_size: {len(train_indices)}")
        print(f"validation_size: {len(val_indices)}")

    if test_dataset is not None:
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        print(f"test_size: {len(test_dataset)}")

    return train_loader, val_loader, test_loader

In [13]:
train_dl, val_dl, _ = create_data_loaders(smap_x_y, BatchSZ)

train_size: 121575
validation_size: 13508


In [14]:
eg = next(iter(val_dl))
eg[0].size(), eg[1].size()

(torch.Size([256, 100, 25]), torch.Size([256, 1, 25]))

In [ ]:
# import sys
# sys.path.insert(0, 'mtad-gat-pytorch/')

In [15]:
import torch
import torch.nn as nn


class ConvLayer(nn.Module):
    """1-D Convolution layer to extract high-level features of each time-series input
    :param n_features: Number of input features/nodes
    :param window_size: length of the input sequence
    :param kernel_size: size of kernel to use in the convolution operation
    """
    def __init__(self, n_features, kernel_size=7):
        super(ConvLayer, self).__init__()
        self.padding = nn.ConstantPad1d((kernel_size - 1) // 2, 0.0)
        self.conv = nn.Conv1d(
            in_channels=n_features, out_channels=n_features, kernel_size=kernel_size
        )
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.padding(x)
        x = self.relu(self.conv(x))
        return x.permute(0, 2, 1)  # Permute back



class FeatureAttentionLayer0(nn.Module):
    """Single Graph Feature/Spatial Attention Layer
    :param n_features: Number of input features/nodes
    :param window_size: length of the input sequence
    :param dropout: percentage of nodes to dropout
    :param alpha: negative slope used in the leaky rely activation function
    :param embed_dim: embedding dimension (output dimension of linear transformation)
    :param use_gatv2: whether to use the modified attention mechanism of GATv2 instead of standard GAT
    :param use_bias: whether to include a bias term in the attention layer
    """

    def __init__(
        self,
        n_features,
        window_size,
        dropout,
        alpha,
        embed_dim=None,
        use_gatv2=True,
        use_bias=True,
    ):
        super(FeatureAttentionLayer0, self).__init__()
        self.n_features = n_features
        self.window_size = window_size
        self.dropout = dropout
        self.embed_dim = embed_dim if embed_dim is not None else window_size
        self.use_gatv2 = use_gatv2
        self.num_nodes = n_features
        self.use_bias = use_bias

        # Because linear transformation is done after concatenation in GATv2
        if self.use_gatv2:
            self.embed_dim *= 2
            lin_input_dim = 2 * window_size
            a_input_dim = self.embed_dim
        else:
            lin_input_dim = window_size
            a_input_dim = 2 * self.embed_dim

        self.lin = nn.Linear(lin_input_dim, self.embed_dim)
        self.a = nn.Parameter(torch.zeros((a_input_dim, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        if self.use_bias:
            self.bias = nn.Parameter(torch.zeros(n_features, n_features))

        self.leakyrelu = nn.LeakyReLU(alpha)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x shape (b, n, k): b - batch size, n - window size, k - number of features
        # C

        x = x.permute(0, 2, 1)

        # 'Dynamic' GAT attention
        # Proposed by Brody et. al., 2021 (https://arxiv.org/pdf/2105.14491.pdf)
        # Linear transformation applied after concatenation and attention layer applied after leakyrelu
        if self.use_gatv2:
            a_input = self._make_attention_input(x)  # (b, k, k, 2*window_size)
            a_input = self.leakyrelu(self.lin(a_input))  # (b, k, k, embed_dim)
            e = torch.matmul(a_input, self.a).squeeze(3)  # (b, k, k, 1)
            

        # Original GAT attention
        else:
            Wx = self.lin(x)  # (b, k, k, embed_dim)
            a_input = self._make_attention_input(Wx)  # (b, k, k, 2*embed_dim)
            e = self.leakyrelu(torch.matmul(a_input, self.a)).squeeze(3)  # (b, k, k, 1)
            

        if self.use_bias:
            e += self.bias

        # Attention weights    
        attention = torch.softmax(e, dim=2)
        attention = torch.dropout(attention, self.dropout, train=self.training)
        # attention = 
        # Computing new node features using the attention
        h = self.sigmoid(torch.matmul(attention, x))
        return h.permute(0, 2, 1)

    def _make_attention_input(self, v):
        """Preparing the feature attention mechanism.
        Creating matrix with all possible combinations of concatenations of node.
        Each node consists of all values of that node within the window
            v1 || v1,
            ...
            v1 || vK,
            v2 || v1,
            ...
            v2 || vK,
            ...
            ...
            vK || v1,
            ...
            vK || vK,
        """

        # K = self.num_nodes
        # blocks_repeating = v.repeat_interleave(K, dim=1)  # Left-side of the matrix    每个元素的重复次数。repeats参数会被广播来适应输入张量的维度
        # blocks_alternating = v.repeat(1, K, 1)  # Right-side of the matrix             沿着特定的维度重复这个张量，和expand()不同的是，这个函数拷贝张量的数据
        # combined = torch.cat(
        #     (blocks_repeating, blocks_alternating), dim=2
        # )  # (b, K*K, 2*window_size)
        K = self.num_nodes
        blocks_repeating = v.repeat_interleave(K, dim=1)  # Left-side of the matrix    每个元素的重复次数。repeats参数会被广播来适应输入张量的维度
        blocks_alternating = v.repeat(1, K, 1)  # Right-side of the matrix             沿着特定的维度重复这个张量，和expand()不同的是，这个函数拷贝张量的数据
        combined = torch.cat(
            (blocks_repeating, blocks_alternating), dim=2
        )  # (b, K*K, 2*window_size)

        if self.use_gatv2:
            return combined.view(v.size(0), K, K, 2 * self.window_size)
        else:
            return combined.view(v.size(0), K, K, 2 * self.embed_dim)
class FeatureAttentionLayer1(nn.Module):
    """Single Graph Feature/Spatial Attention Layer
    :param n_features: Number of input features/nodes
    :param window_size: length of the input sequence
    :param dropout: percentage of nodes to dropout
    :param alpha: negative slope used in the leaky rely activation function
    :param embed_dim: embedding dimension (output dimension of linear transformation)
    :param use_gatv2: whether to use the modified attention mechanism of GATv2 instead of standard GAT
    :param use_bias: whether to include a bias term in the attention layer
    """

    def __init__(
        self,
        n_features,
        window_size,
        dropout,
        alpha,
        embed_dim=None,
        use_gatv2=True,
        use_bias=True,
    ):
        super(FeatureAttentionLayer1, self).__init__()
        self.n_features = n_features
        self.window_size = window_size
        self.dropout = dropout
        self.embed_dim = embed_dim if embed_dim is not None else window_size
        self.use_gatv2 = use_gatv2
        self.num_nodes = n_features
        self.use_bias = use_bias

        # Because linear transformation is done after concatenation in GATv2
        if self.use_gatv2:
            self.embed_dim *= 2
            lin_input_dim = 2 * window_size
            a_input_dim = self.embed_dim
        else:
            lin_input_dim = window_size
            a_input_dim = 2 * self.embed_dim

        self.lin = nn.Linear(lin_input_dim, self.embed_dim)
        self.a = nn.Parameter(torch.rand((a_input_dim, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        if self.use_bias:
            self.bias = nn.Parameter(torch.rand(n_features, n_features))

        self.leakyrelu = nn.LeakyReLU(alpha)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x shape (b, n, k): b - batch size, n - window size, k - number of features
        # C

        x = x.permute(0, 2, 1)

        # 'Dynamic' GAT attention
        # Proposed by Brody et. al., 2021 (https://arxiv.org/pdf/2105.14491.pdf)
        # Linear transformation applied after concatenation and attention layer applied after leakyrelu
        if self.use_gatv2:
            a_input = self._make_attention_input(x)  # (b, k, k, 2*window_size)
            a_input = self.leakyrelu(self.lin(a_input))  # (b, k, k, embed_dim)
            e = torch.matmul(a_input, self.a).squeeze(3)  # (b, k, k, 1)
            

        # Original GAT attention
        else:
            Wx = self.lin(x)  # (b, k, k, embed_dim)
            a_input = self._make_attention_input(Wx)  # (b, k, k, 2*embed_dim)
            e = self.leakyrelu(torch.matmul(a_input, self.a)).squeeze(3)  # (b, k, k, 1)
            

        if self.use_bias:
            e += self.bias

        # Attention weights    
        attention = torch.softmax(e, dim=2)
        attention = torch.dropout(attention, self.dropout, train=self.training)
        # attention = 
        # Computing new node features using the attention
        h = self.sigmoid(torch.matmul(attention, x))
        return h.permute(0, 2, 1)

    def _make_attention_input(self, v):
        """Preparing the feature attention mechanism.
        Creating matrix with all possible combinations of concatenations of node.
        Each node consists of all values of that node within the window
            v1 || v1,
            ...
            v1 || vK,
            v2 || v1,
            ...
            v2 || vK,
            ...
            ...
            vK || v1,
            ...
            vK || vK,
        """

        # K = self.num_nodes
        # blocks_repeating = v.repeat_interleave(K, dim=1)  # Left-side of the matrix    每个元素的重复次数。repeats参数会被广播来适应输入张量的维度
        # blocks_alternating = v.repeat(1, K, 1)  # Right-side of the matrix             沿着特定的维度重复这个张量，和expand()不同的是，这个函数拷贝张量的数据
        # combined = torch.cat(
        #     (blocks_repeating, blocks_alternating), dim=2
        # )  # (b, K*K, 2*window_size)
        K = self.num_nodes
        blocks_repeating = v.repeat_interleave(K, dim=1)  # Left-side of the matrix    每个元素的重复次数。repeats参数会被广播来适应输入张量的维度
        blocks_alternating = v.repeat(1, K, 1)  # Right-side of the matrix             沿着特定的维度重复这个张量，和expand()不同的是，这个函数拷贝张量的数据
        combined = torch.cat(
            (blocks_repeating, blocks_alternating), dim=2
        )  # (b, K*K, 2*window_size)

        if self.use_gatv2:
            return combined.view(v.size(0), K, K, 2 * self.window_size)
        else:
            return combined.view(v.size(0), K, K, 2 * self.embed_dim)


class TemporalAttentionLayer(nn.Module):
    """Single Graph Temporal Attention Layer
    :param n_features: number of input features/nodes
    :param window_size: length of the input sequence
    :param dropout: percentage of nodes to dropout
    :param alpha: negative slope used in the leaky rely activation function
    :param embed_dim: embedding dimension (output dimension of linear transformation)
    :param use_gatv2: whether to use the modified attention mechanism of GATv2 instead of standard GAT
    :param use_bias: whether to include a bias term in the attention layer

    """

    def __init__(
        self,
        n_features,
        window_size,
        dropout,
        alpha,
        embed_dim=None,
        use_gatv2=True,
        use_bias=True,
    ):
        super(TemporalAttentionLayer, self).__init__()
        self.n_features = n_features
        self.window_size = window_size
        self.dropout = dropout
        self.use_gatv2 = use_gatv2
        self.embed_dim = embed_dim if embed_dim is not None else n_features
        self.num_nodes = window_size
        self.use_bias = use_bias

        # Because linear transformation is performed after concatenation in GATv2
        if self.use_gatv2:
            self.embed_dim *= 2
            lin_input_dim = 2 * n_features
            a_input_dim = self.embed_dim
        else:
            lin_input_dim = n_features
            a_input_dim = 2 * self.embed_dim

        self.lin = nn.Linear(lin_input_dim, self.embed_dim)
        self.a = nn.Parameter(torch.zeros((a_input_dim, 1)))
        #self.a = nn.Parameter(torch.rand((a_input_dim, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        if self.use_bias:
            self.bias = nn.Parameter(torch.zeros(window_size, window_size))

        self.leakyrelu = nn.LeakyReLU(alpha)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x shape (b, n, k): b - batch size, n - window size, k - number of features
        # For temporal attention a node is represented as all feature values at a specific timestamp

        # 'Dynamic' GAT attention
        # Proposed by Brody et. al., 2021 (https://arxiv.org/pdf/2105.14491.pdf)
        # Linear transformation applied after concatenation and attention layer applied after leakyrelu
        if self.use_gatv2:
            a_input = self._make_attention_input(x)  # (b, n, n, 2*n_features)
            a_input = self.leakyrelu(self.lin(a_input))  # (b, n, n, embed_dim)
            e = torch.matmul(a_input, self.a).squeeze(3)  # (b, n, n, 1)

        # Original GAT attention
        else:
            Wx = self.lin(x)  # (b, n, n, embed_dim)
            a_input = self._make_attention_input(Wx)  # (b, n, n, 2*embed_dim)
            e = self.leakyrelu(torch.matmul(a_input, self.a)).squeeze(3)  # (b, n, n, 1)

        if self.use_bias:
            e += self.bias  # (b, n, n, 1)

        # Attention weights
        attention = torch.softmax(e, dim=2)
        attention = torch.dropout(attention, self.dropout, train=self.training)

        h = self.sigmoid(torch.matmul(attention, x))  # (b, n, k)
        return h

    def _make_attention_input(self, v):
        """Preparing the temporal attention mechanism.
        Creating matrix with all possible combinations of concatenations of node values:
            (v1, v2..)_t1 || (v1, v2..)_t1
            (v1, v2..)_t1 || (v1, v2..)_t2

            ...
            ...

            (v1, v2..)_tn || (v1, v2..)_t1
            (v1, v2..)_tn || (v1, v2..)_t2

        """

        K = self.num_nodes
        blocks_repeating = v.repeat_interleave(K, dim=1)  # Left-side of the matrix

        blocks_alternating = v.repeat(1, K, 1)  # Right-side of the matrix
        combined = torch.cat((blocks_repeating, blocks_alternating), dim=2)

        

        if self.use_gatv2:
            return combined.view(v.size(0), K, K, 2 * self.n_features)
        else:
            return combined.view(v.size(0), K, K, 2 * self.embed_dim)


# class GRULayer(nn.Module):
#     """Gated Recurrent Unit (GRU) Layer
#     :param in_dim: number of input features
#     :param hid_dim: hidden size of the GRU
#     :param n_layers: number of layers in GRU
#     :param dropout: dropout rate
#     """

#     def __init__(self, in_dim, hid_dim, n_layers, dropout):
#         super(GRULayer, self).__init__()
#         self.hid_dim = hid_dim
#         self.n_layers = n_layers
#         self.dropout = 0.0 if n_layers == 1 else dropout
#         self.gru = nn.GRU(
#             in_dim, hid_dim, num_layers=n_layers, batch_first=True, dropout=self.dropout
#         )

#     def forward(self, x):
#         out, h = self.gru(x)
#         out, h = out[-1, :, :], h[-1, :, :]  # Extracting from last layer
#         return out, h

class LSTMLayer(nn.Module):
    """Gated Recurrent Unit (GRU) Layer
    :param in_dim: number of input features
    :param hid_dim: hidden size of the GRU
    :param n_layers: number of layers in GRU
    :param dropout: dropout rate
    """

    def __init__(self, in_dim, hid_dim, n_layers, dropout):
        super(LSTMLayer, self).__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.dropout = 0.0 if n_layers == 1 else dropout
        self.lstm = nn.LSTM(
            in_dim, hid_dim, num_layers=n_layers, batch_first=True, dropout=self.dropout
        )

    def forward(self, x):
        out, (h, c) = self.lstm(x)
        out, h = out[-1, :, :], h[-1, :, :]  # Extracting from last layer
        return out, h



# class RNNDecoder(nn.Module):
#     """GRU-based Decoder network that converts latent vector into output
#     :param in_dim: number of input features
#     :param n_layers: number of layers in RNN
#     :param hid_dim: hidden size of the RNN
#     :param dropout: dropout rate
#     """

#     def __init__(self, in_dim, hid_dim, n_layers, dropout):
#         super(RNNDecoder, self).__init__()
#         self.in_dim = in_dim
#         self.dropout = 0.0 if n_layers == 1 else dropout
#         self.rnn = nn.GRU(
#             in_dim, hid_dim, n_layers, batch_first=True, dropout=self.dropout
#         )

#     def forward(self, x):
#         decoder_out, _ = self.rnn(x)
#         return decoder_out
    
class LSTMDecoder(nn.Module):
    """LSTM-based Decoder network that converts latent vector into output
    :param in_dim: number of input features
    :param n_layers: number of layers in RNN
    :param hid_dim: hidden size of the RNN
    :param dropout: dropout rate
    """

    def __init__(self, in_dim, hid_dim, n_layers, dropout):
        super(LSTMDecoder, self).__init__()
        self.in_dim = in_dim
        self.dropout = 0.0 if n_layers == 1 else dropout
        self.lstm= nn.LSTM(
            in_dim, hid_dim, n_layers, batch_first=True, dropout=self.dropout
        )

    def forward(self, x):
        decoder_out, _ = self.lstm(x)
        return decoder_out



class ReconstructionModel(nn.Module):
    """Reconstruction Model
    :param window_size: length of the input sequence
    :param in_dim: number of input features
    :param n_layers: number of layers in RNN
    :param hid_dim: hidden size of the RNN
    :param in_dim: number of output features
    :param dropout: dropout rate
    """

    def __init__(self, window_size, in_dim, hid_dim, out_dim, n_layers, dropout):
        super(ReconstructionModel, self).__init__()
        self.window_size = window_size
        self.decoder = LSTMDecoder(in_dim, hid_dim, n_layers, dropout)
        self.fc = nn.Linear(hid_dim, out_dim)
        self.sigma1 = nn.Parameter(torch.tensor(0.5))

    def forward(self, x):
        # x will be last hidden state of the GRU layer
        h_end = x
        h_end_rep = h_end.repeat_interleave(self.window_size, dim=1).view(
            x.size(0), self.window_size, -1
        )

        decoder_out = self.decoder(h_end_rep)
        out = self.fc(decoder_out)
        out = self.sigma1 * out
        return out


class Forecasting_Model(nn.Module):
    """Forecasting model (fully-connected network)
    :param in_dim: number of input features
    :param hid_dim: hidden size of the FC network
    :param out_dim: number of output features
    :param n_layers: number of FC layers
    :param dropout: dropout rate
    """

    def __init__(self, in_dim, hid_dim, out_dim, n_layers, dropout):
        super(Forecasting_Model, self).__init__()
        layers = [nn.Linear(in_dim, hid_dim)]
        for _ in range(n_layers - 1):
            layers.append(nn.Linear(hid_dim, hid_dim))

        layers.append(nn.Linear(hid_dim, out_dim))

        self.layers = nn.ModuleList(layers)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.sigma2 = nn.Parameter(torch.tensor(0.5))

    def forward(self, x):
        for i in range(len(self.layers) - 1):
            x = self.relu(self.layers[i](x))
            x = self.dropout(x)
            x = self.sigma2 * x
        return self.layers[-1](x)


In [16]:
class MTAD_GAT(nn.Module):

    def __init__(
        self,
        n_features,
        window_size,
        out_dim,
        kernel_size=7,
        feat_gat_embed_dim=None,
        time_gat_embed_dim=None,
        use_gatv2=True,
        gru_n_layers=1,
        gru_hid_dim=150,
        forecast_n_layers=1,
        forecast_hid_dim=150,
        recon_n_layers=1,
        recon_hid_dim=150,
        dropout=0.2,
        alpha=0.2
    ):
        super(MTAD_GAT, self).__init__()

        self.conv = ConvLayer(n_features, kernel_size)
        self.feature_gat_0 = FeatureAttentionLayer0(n_features, window_size, dropout, alpha, feat_gat_embed_dim, use_gatv2= False , use_bias = False)
        self.feature_gat_1 = FeatureAttentionLayer1(n_features, window_size, dropout, alpha, feat_gat_embed_dim, use_gatv2 , use_bias = False)
        #self.feature_gat_2 = FeatureAttentionLayer0(n_features, window_size, dropout, alpha, feat_gat_embed_dim, use_gatv2 , use_bias = False)
        #self.feature_gat_3 = FeatureAttentionLayer1(n_features, window_size, dropout, alpha, feat_gat_embed_dim, use_gatv2= False , use_bias = False)
        self.sigmoid = nn.Sigmoid()
        self.temporal_gat_0 = TemporalAttentionLayer(n_features, window_size, dropout, alpha, time_gat_embed_dim, use_gatv2, use_bias = False)
#         self.temporal_gat_1 = TemporalAttentionLayer(n_features, window_size, dropout, alpha, time_gat_embed_dim, use_gatv2, use_bias = False)
#         self.sigmoid = nn.Sigmoid()
        
        self.lstm = LSTMLayer(3 * n_features, gru_hid_dim, gru_n_layers, dropout)
        #self.gru = GRULayer(3 * n_features, gru_hid_dim, gru_n_layers, dropout)
       
        self.forecasting_model = Forecasting_Model(gru_hid_dim, forecast_hid_dim, out_dim, forecast_n_layers, dropout)
        self.recon_model = ReconstructionModel(window_size, gru_hid_dim, recon_hid_dim, out_dim, recon_n_layers, dropout)
        

    def forward(self, x):
        # x shape (b, n, k): b - batch size, n - window size, k - number of features

        x = self.conv(x)
        h_feat_0 = self.feature_gat_0(x)
        h_feat_1 = self.feature_gat_1(x)
#         h_feat_2 = self.feature_gat_2(x)
#         h_feat_3 = self.feature_gat_3(x)
        h_feat = self.sigmoid(h_feat_0 + h_feat_1)
        h_temp = self.temporal_gat_0(x)
#         h_temp_1 = self.temporal_gat_1(x)
#         h_temp = self.sigmoid(h_temp_1 + h_temp_0)
        h_cat = torch.cat([x, h_feat_0, h_temp], dim=2)  # (b, n, 3k)
        _, h_end = self.lstm(h_cat)
        h_end = h_end.view(x.shape[0], -1)   # Hidden state for last timestamp

        predictions = self.forecasting_model(h_end)
        recons = self.recon_model(h_end)

        return predictions, recons

In [17]:
model = MTAD_GAT(n_features = 25,  window_size = 100, out_dim = 25, feat_gat_embed_dim = 256, time_gat_embed_dim = 64)
model.to(device)

MTAD_GAT(
  (conv): ConvLayer(
    (padding): ConstantPad1d(padding=(3, 3), value=0.0)
    (conv): Conv1d(25, 25, kernel_size=(7,), stride=(1,))
    (relu): ReLU()
  )
  (feature_gat_0): FeatureAttentionLayer0(
    (lin): Linear(in_features=100, out_features=256, bias=True)
    (leakyrelu): LeakyReLU(negative_slope=0.2)
    (sigmoid): Sigmoid()
  )
  (feature_gat_1): FeatureAttentionLayer1(
    (lin): Linear(in_features=200, out_features=512, bias=True)
    (leakyrelu): LeakyReLU(negative_slope=0.2)
    (sigmoid): Sigmoid()
  )
  (sigmoid): Sigmoid()
  (temporal_gat_0): TemporalAttentionLayer(
    (lin): Linear(in_features=50, out_features=128, bias=True)
    (leakyrelu): LeakyReLU(negative_slope=0.2)
    (sigmoid): Sigmoid()
  )
  (lstm): LSTMLayer(
    (lstm): LSTM(75, 150, batch_first=True)
  )
  (forecasting_model): Forecasting_Model(
    (layers): ModuleList(
      (0): Linear(in_features=150, out_features=150, bias=True)
      (1): Linear(in_features=150, out_features=25, bias=Tr

In [18]:
eg_out = model(eg[0].float().to(device))
eg_out[0].size(), eg_out[1].size()

(torch.Size([256, 25]), torch.Size([256, 100, 25]))

In [19]:
EPOCHS =  10
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
forecast_criterion = nn.MSELoss()
#weighted_criterion0 = lambda output, target: forecast_criterion(output, target) * self.alpha
recon_criterion = nn.MSELoss()
#weighted_criterion1 = lambda output, target: recon_criterion(output, target) * self.alpha
len(train_dl), len(val_dl)

(475, 53)

In [20]:
!mkdir model_checkpoints

In [21]:
Best_Valid_loss = 999999
Best_Epoch = -1

for epoch in range(EPOCHS):
    model.train()
    forecast_b_losses = []
    recon_b_losses = []
    
    print("Epoch : ", epoch)
    print("Training Started ... ")
    
    for batch_idx, batch in enumerate(train_dl):
        x = batch[0].to(device).float()
        y = batch[1].to(device).float()
        optimizer.zero_grad()

        preds, recons = model(x)

        forecast_loss = torch.sqrt(forecast_criterion(y.squeeze(1), preds))
        recon_loss = torch.sqrt(recon_criterion(x, recons))
        loss = forecast_loss + recon_loss

        loss.backward()
        optimizer.step()

        forecast_b_losses.append(forecast_loss.item())
        recon_b_losses.append(recon_loss.item())
        

    forecast_b_losses = np.array(forecast_b_losses)
    recon_b_losses = np.array(recon_b_losses)

    forecast_epoch_loss = np.sqrt((forecast_b_losses ** 2).mean())
    recon_epoch_loss = np.sqrt((recon_b_losses ** 2).mean())

    total_epoch_loss = forecast_epoch_loss + recon_epoch_loss
    
    print('Forecasting Loss : ', forecast_epoch_loss)
    print('Reconstruction Loss : ', recon_epoch_loss)
    
    
    forecast_b_losses_eval = []
    recon_b_losses_eval = []
    
    print("Validation Started ... ")
    
    model.eval()
    with torch.no_grad():
        for batch_idx, batch in enumerate(val_dl):
            x = batch[0].to(device).float()
            y = batch[1].to(device).float()

            preds, recons = model(x)

            forecast_loss = torch.sqrt(forecast_criterion(y.squeeze(1), preds))
            recon_loss = torch.sqrt(recon_criterion(x, recons))
            loss = forecast_loss + recon_loss

            forecast_b_losses_eval.append(forecast_loss.item())
            recon_b_losses_eval.append(recon_loss.item())

    forecast_b_losses_eval = np.array(forecast_b_losses_eval)
    recon_b_losses_eval = np.array(recon_b_losses_eval)

    forecast_epoch_loss_eval = np.sqrt((forecast_b_losses_eval ** 2).mean())
    recon_epoch_loss_eval = np.sqrt((recon_b_losses_eval ** 2).mean())

    total_epoch_loss_eval = 0.5*forecast_epoch_loss_eval + 0.5*recon_epoch_loss_eval
    
    print('Forecasting Loss : ', forecast_epoch_loss_eval)
    print('Reconstruction Loss : ', recon_epoch_loss_eval)
    
    if total_epoch_loss_eval < Best_Valid_loss:
        Best_Valid_loss = total_epoch_loss_eval
        Best_Epoch = epoch
        
        ckpt = {
            'Epoch' : epoch,
            'Model' : model.state_dict(),
            'Optimizer' : optimizer.state_dict(),
            'Train_Forecast_loss' : forecast_epoch_loss,
            'Train_Recon_loss' : recon_epoch_loss,
            'Eval_Forecast_loss' : forecast_epoch_loss_eval,
            'Eval_Recon_loss' : recon_epoch_loss_eval
        }
        
        torch.save(ckpt, os.path.join('model_checkpoints', str(epoch) + '.pt'))

Epoch :  0
Training Started ... 
Forecasting Loss :  0.10773431934925987
Reconstruction Loss :  0.1310970797970022
Validation Started ... 
Forecasting Loss :  0.0864743479052625
Reconstruction Loss :  0.10197424094906253
Epoch :  1
Training Started ... 
Forecasting Loss :  0.0876810899182744
Reconstruction Loss :  0.09582280989872405
Validation Started ... 
Forecasting Loss :  0.08159810944897217
Reconstruction Loss :  0.09186539559388296
Epoch :  2
Training Started ... 
Forecasting Loss :  0.08473780460725663
Reconstruction Loss :  0.08947709128513345
Validation Started ... 
Forecasting Loss :  0.0808050693384635
Reconstruction Loss :  0.08877213161734335
Epoch :  3
Training Started ... 
Forecasting Loss :  0.08328809623990593
Reconstruction Loss :  0.08568393347258783
Validation Started ... 
Forecasting Loss :  0.0787444608071155
Reconstruction Loss :  0.09047321656181047
Epoch :  4
Training Started ... 
Forecasting Loss :  0.08238312103758723
Reconstruction Loss :  0.083333644583737

In [22]:
best_model = MTAD_GAT(n_features = 25, window_size = 100, out_dim = 25, feat_gat_embed_dim = 256, time_gat_embed_dim = 64)
best_model.load_state_dict( torch.load( os.path.join( 'model_checkpoints', str(Best_Epoch) + '.pt' ) )['Model'] )
best_model.to(device)

MTAD_GAT(
  (conv): ConvLayer(
    (padding): ConstantPad1d(padding=(3, 3), value=0.0)
    (conv): Conv1d(25, 25, kernel_size=(7,), stride=(1,))
    (relu): ReLU()
  )
  (feature_gat_0): FeatureAttentionLayer0(
    (lin): Linear(in_features=100, out_features=256, bias=True)
    (leakyrelu): LeakyReLU(negative_slope=0.2)
    (sigmoid): Sigmoid()
  )
  (feature_gat_1): FeatureAttentionLayer1(
    (lin): Linear(in_features=200, out_features=512, bias=True)
    (leakyrelu): LeakyReLU(negative_slope=0.2)
    (sigmoid): Sigmoid()
  )
  (sigmoid): Sigmoid()
  (temporal_gat_0): TemporalAttentionLayer(
    (lin): Linear(in_features=50, out_features=128, bias=True)
    (leakyrelu): LeakyReLU(negative_slope=0.2)
    (sigmoid): Sigmoid()
  )
  (lstm): LSTMLayer(
    (lstm): LSTM(75, 150, batch_first=True)
  )
  (forecasting_model): Forecasting_Model(
    (layers): ModuleList(
      (0): Linear(in_features=150, out_features=150, bias=True)
      (1): Linear(in_features=150, out_features=25, bias=Tr

In [23]:
smap_data_test = []
for smap_channel in smap_channels:
    tmp_data = np.load(os.path.join('data/test/', smap_channel + '.npy'))
    smap_data_test.extend(tmp_data)
    
smap_data_test = np.array(smap_data_test)
print("Shape of Test DataSet : ", smap_data_test.shape)

Shape of Test DataSet :  (427617, 25)


In [24]:
smap_data_test_norm, _ = normalize_data(smap_data_test, scaler)
smap_data_test_norm = smap_data_test_norm[:10000]
smap_data_test_pt = torch.from_numpy(smap_data_test_norm)
smap_data_test_pt.size()

Data normalized


torch.Size([10000, 25])

In [25]:
smap_test_x_y = SlidingWindowDataset(smap_data_test_pt, 100)
test_dl = torch.utils.data.DataLoader(smap_test_x_y, batch_size=256, shuffle=False)
print("Number of Batches in Test Dataloader : ", len(test_dl))

Number of Batches in Test Dataloader :  39


In [26]:
model.eval()
preds = []
recons = []

with torch.no_grad():
    for batch in tqdm(test_dl):
        x = batch[0].to(device).float()
        y = batch[1].to(device).float()

        y_hat, _ = model(x)

        # Shifting input to include the observed value (y) when doing the reconstruction
        recon_x = torch.cat((x[:, 1:, :], y), dim=1)
        _, window_recon = model(recon_x)

        preds.append(y_hat.detach().cpu().numpy())
        # Extract last reconstruction only
        recons.append(window_recon[:, -1, :].detach().cpu().numpy())

    preds = np.concatenate(preds, axis=0)
    recons = np.concatenate(recons, axis=0)

  0%|          | 0/39 [00:00<?, ?it/s]

In [27]:
preds.shape, recons.shape
scale_scores = True

In [28]:
actual = smap_data_test_norm[Window:]
print(actual.shape)

anomaly_scores = np.zeros_like(actual)
df = pd.DataFrame()
for i in range(preds.shape[1]):
    df[f"Forecast_{i}"] = preds[:, i]
    df[f"Recon_{i}"] = recons[:, i]
    df[f"True_{i}"] = actual[:, i]
    a_score = np.sqrt((preds[:, i] - actual[:, i]) ** 2) + np.sqrt(
        (recons[:, i] - actual[:, i]) ** 2)

    if scale_scores:
        q75, q25 = np.percentile(a_score, [75, 25])
        iqr = q75 - q25
        median = np.median(a_score)
        a_score = (a_score - median) / (1+iqr)

    anomaly_scores[:, i] = a_score
    df[f"A_Score_{i}"] = a_score

anomaly_scores = np.mean(anomaly_scores, 1)
df['A_Score_Global'] = anomaly_scores

(9900, 25)


<ipython-input-28-bbbe538e9e64>:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['A_Score_Global'] = anomaly_scores


In [29]:
csv_data = df.to_csv('score_1.csv', index = True)
df

,Forecast_0,Recon_0,True_0,A_Score_0,Forecast_1,Recon_1,True_1,A_Score_1,Forecast_2,Recon_2,...,A_Score_22,Forecast_23,Recon_23,True_23,A_Score_23,Forecast_24,Recon_24,True_24,A_Score_24,A_Score_Global
0,0.952727,0.960019,1.000000,-0.005868,0.009776,-0.001783,0.0,-0.004031,0.000511,0.000926,...,-0.001784,-1.989262e-06,0.001098,0.0,0.000211,-3.414730e-10,0.001461,0.0,0.000163,-0.001321
1,0.955124,0.960135,1.000000,-0.008282,0.009993,-0.001620,0.0,-0.003978,0.000567,0.000857,...,-0.001976,-1.963720e-06,0.001097,0.0,0.000211,-2.448125e-10,0.001468,0.0,0.000170,-0.001421
2,0.954229,0.960200,1.000000,-0.007484,0.010110,-0.001354,0.0,-0.004122,0.000579,0.000747,...,-0.002296,-1.926468e-06,0.001097,0.0,0.000210,-2.094999e-10,0.001475,0.0,0.000177,-0.001467
3,0.956674,0.959991,1.000000,-0.009632,0.010367,-0.001207,0.0,-0.004016,0.000600,0.000590,...,-0.002576,-1.874917e-06,0.001094,0.0,0.000208,-1.695092e-10,0.001476,0.0,0.000178,-0.001587
4,0.955022,0.958941,1.000000,-0.007037,0.010401,-0.001279,0.0,-0.003913,0.000585,0.000389,...,-0.002968,-1.834807e-06,0.001088,0.0,0.000201,-1.644644e-10,0.001459,0.0,0.000160,-0.001587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9895,0.337292,-0.115536,0.144039,0.345199,0.001096,0.155344,0.0,0.136169,-0.000838,0.001816,...,0.006198,-8.049562e-07,-0.000356,0.0,-0.000531,3.468313e-09,0.000413,0.0,-0.000884,0.031157
9896,0.197003,-0.003861,0.144039,0.103234,-0.000650,0.053595,0.0,0.037277,-0.002960,-0.000270,...,-0.002206,-7.075462e-07,-0.000334,0.0,-0.000553,1.343798e-09,0.000454,0.0,-0.000843,0.006769
9897,0.113234,0.132231,0.144039,-0.048738,-0.002300,0.037999,0.0,0.023781,-0.003321,-0.001472,...,-0.000044,-9.843636e-07,-0.000272,0.0,-0.000615,3.906150e-10,0.000499,0.0,-0.000799,-0.002129
9898,0.131095,0.202282,0.139819,-0.021297,-0.001897,0.030896,0.0,0.016518,-0.003227,-0.002546,...,0.001757,-1.160013e-06,-0.000227,0.0,-0.000659,-4.501588e-11,0.000511,0.0,-0.000787,-0.001046
